# Setup

In [1]:
from pymongo import MongoClient

In [2]:
db = MongoClient().aggregation_example

In [3]:
result = db.things.insert_many([{'x': 1, 'tags': ['dog', 'cat']},
                                {'x': 2, 'tags': ['cat']},
                                {'x': 2, 'tags': ['mouse', 'cat', 'dog']},
                                {'x': 3, 'tags': []}])

In [4]:
result.inserted_ids

[ObjectId('59a7ba3b498ab10af626688c'),
 ObjectId('59a7ba3b498ab10af626688d'),
 ObjectId('59a7ba3b498ab10af626688e'),
 ObjectId('59a7ba3b498ab10af626688f')]

# Aggregation Framework

In [5]:
from bson.son import SON

In [6]:
pipeline = [
    {'$unwind': '$tags'},
    {'$group': {'_id': '$tags', 'count': {'$sum': 1}}},
    {'$sort': SON([('count', -1), ('_id', -1)])}
]

In [7]:
import pprint

In [8]:
pprint.pprint(list(db.things.aggregate(pipeline)))

[{'_id': 'cat', 'count': 6},
 {'_id': 'dog', 'count': 4},
 {'_id': 'mouse', 'count': 2}]


In [11]:
db.command('aggregate', 'things', pipeline=pipeline, explain=True)

{'ok': 1.0,
 'stages': [{'$cursor': {'fields': {'_id': 0, 'tags': 1},
    'query': {},
    'queryPlanner': {'indexFilterSet': False,
     'namespace': 'aggregation_example.things',
     'parsedQuery': {},
     'plannerVersion': 1,
     'rejectedPlans': [],
     'winningPlan': {'direction': 'forward', 'stage': 'COLLSCAN'}}}},
  {'$unwind': {'path': '$tags'}},
  {'$group': {'_id': '$tags', 'count': {'$sum': {'$const': 1}}}},
  {'$sort': {'sortKey': {'_id': -1, 'count': -1}}}]}

# Map/Reduce